In [1]:
import src.test_functions as test_functions

In [2]:
import scipy
import numpy as np

In [3]:
import matplotlib.pyplot as plt

In [4]:
import os
os.environ['XLA_PYTHON_CLIENT_MEM_FRACTION']='.1'

In [5]:
import torch.nn as nn
import torch

from tqdm import tqdm

In [6]:
from sympy import symbols, Max
import sympy

In [7]:
from pyibex import Interval, IntervalVector, Function, CtcFwdBwd, SepFwdBwd, GEQ

In [8]:
seed = 0

import random
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

##  Sample points for training

In [9]:
def sample(function, bounds, n, local_minima=False):
    assert isinstance(bounds, np.ndarray)
    assert bounds.shape[1] == 2
    xs = np.random.rand(n, bounds.shape[0]) * (bounds[:, 1] - bounds[:, 0]) + bounds[:, 0]
    ys = np.array([function(x).item() for x in xs])
    
    if local_minima:
        xs_local_opt = []
        ys_local_opt = []
        for x0 in xs:
            opt_res = scipy.optimize.minimize(func, x0, method='Nelder-Mead')
            if opt_res.success:
                xs_local_opt.append(opt_res.x)
                ys_local_opt.append(opt_res.fun)
        xs_local_opt, ys_local_opt = np.array(xs_local_opt), np.array(ys_local_opt)
        return xs, ys, xs_local_opt, ys_local_opt
    
    return xs, ys

In [10]:
dimension = 1

func = test_functions.Ackley(dims=dimension)
# variables, expression = func.expression()

bounds = func.get_default_domain()
lb = bounds.T[0]
ub = bounds.T[1]

In [11]:
xs = np.arange(lb[0], ub[0], 0.01)
ys = np.array([func(np.array([x])).item() for x in xs])

with jax.default_device(jax.devices("cpu")[0]):
    # sampled_xs, sampled_ys = sample(func, bounds, 100)
    sampled_xs, sampled_ys, sampled_xs_local_min, sampled_ys_local_min = sample(func, bounds, 100, local_minima=True)

2023-05-05 17:27:07.795855: E external/xla/xla/stream_executor/cuda/cuda_dnn.cc:417] Loaded runtime CuDNN library: 8.7.0 but source was compiled with: 8.8.0.  CuDNN library needs to have matching major version and equal or higher minor version. If using a binary install, upgrade your CuDNN library.  If building from sources, make sure the library loaded at runtime is compatible with the version specified during compile configuration.


XlaRuntimeError: FAILED_PRECONDITION: DNN library initialization failed. Look at the errors above for more details.

In [ ]:
fig = plt.figure(figsize=(10,8))
plt.scatter(sampled_xs[:, 0], sampled_ys, s=15, c='red', zorder=10)
# plt.scatter(sampled_xs_local_min[:, 0], sampled_ys_local_min, s=15, c='red', zorder=10)

plt.plot(xs, ys, zorder=1, linewidth=1, alpha=0.7)


In [ ]:
fig = plt.figure(figsize=(10,8))
# plt.scatter(sampled_xs[:, 0], sampled_ys, s=15, c='red', zorder=10)
plt.scatter(sampled_xs_local_min[:, 0], sampled_ys_local_min, s=15, c='red', zorder=10)

plt.plot(xs, ys, zorder=1, linewidth=1, alpha=0.7)


## Fit a neural network

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
xs_train = sampled_xs
ys_train = np.expand_dims(sampled_ys, axis=1)

In [ ]:
hidden_dim = 16

model = nn.Sequential(
    nn.Linear(dimension, hidden_dim),
    nn.ReLU(),
    nn.Linear(hidden_dim, 1)
)

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [ ]:
num_epochs = 10000

model.to(device)

loss_record = []
for epoch in tqdm(range(num_epochs)):
    model.train()
    
    x_train_tensor = torch.FloatTensor(xs_train).to(device)
    y_train_tensor = torch.FloatTensor(ys_train).to(device)
    y_pred = model(x_train_tensor)
    loss = criterion(y_pred, y_train_tensor)
    
    loss_record.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    
    optimizer.step()

In [ ]:
with torch.no_grad():
    fitted_ys = model(torch.FloatTensor(np.expand_dims(xs, axis=1)).to(device)).detach().cpu().squeeze().numpy()

In [ ]:
abs(ys - fitted_ys)

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))

ax.scatter(xs_train[:, 0], ys_train[:, 0], s=15, c='red', zorder=10, label="samples")
# plt.scatter(sampled_xs_local_min[:, 0], sampled_ys_local_min, s=15, c='red', zorder=10)

ax.plot(xs, ys, zorder=1, linewidth=1, alpha=0.7, label="original_function")
ax.plot(xs, fitted_ys, linewidth=1, alpha=0.7, label="fitted_function")

y_diff_squared = (ys - fitted_ys) ** 2
ax.plot(xs, y_diff_squared, linewidth=1, alpha=0.4, label="squared_diff")

y_diff_abs = abs(ys - fitted_ys)
ax.plot(xs, y_diff_abs, linewidth=1, alpha=0.4, label="abs_diff")

ax.legend()

In [ ]:
fig = plt.figure(figsize=(10,8))

plt.scatter(xs_train[:, 0], ys_train[:, 0], s=15, c='red', zorder=10)
# plt.scatter(sampled_xs_local_min[:, 0], sampled_ys_local_min, s=15, c='red', zorder=10)

y_diff_squared = (ys - fitted_ys) ** 2
plt.plot(xs, y_diff_squared, linewidth=1, alpha=0.7)

## Convert the neural network to ibex expression (through sympy) 

In [ ]:
layer_idx = 2
layer = model[layer_idx]

In [ ]:
weight = layer.weight
bias = layer.bias
width = weight.shape[1]

In [ ]:
inter_vars = symbols(', '.join(['x[{}][{}]'.format(layer_idx, i) for i in range(width)]))

In [ ]:
inter_vars

In [ ]:
X = np.expand_dims(np.array(inter_vars), axis=1)

In [ ]:
W = weight.detach().cpu().numpy()

In [ ]:
b = bias.detach().cpu().numpy()
b

In [ ]:
expression = (W @ X).squeeze(axis=1) + b
expression = expression.item()

In [ ]:
layer_idx = 1
layer = model[layer_idx]
layer

In [ ]:
prev_inter_vars = inter_vars
width = len(inter_vars)
inter_vars = symbols(', '.join(['x[{}][{}]'.format(layer_idx, i) for i in range(width)]))

In [ ]:
inter_vars[0]

In [ ]:
replace_dict = {}
for curr_var, prev_var in zip(inter_vars, prev_inter_vars):
    replace_dict[prev_var] = Max(curr_var, 0)

In [ ]:
prev_expression = expression
for key in replace_dict:
    expression = expression.subs(key, replace_dict[key])

In [ ]:
expression

In [ ]:
layer_idx = 0
layer = model[layer_idx]
layer

In [ ]:
weight = layer.weight
bias = layer.bias
width = weight.shape[1]

In [ ]:
prev_inter_vars = inter_vars
inter_vars = symbols(', '.join(['x[{}]'.format(i) for i in range(width)]))
if isinstance(inter_vars, sympy.Symbol):
    inter_vars = [inter_vars]

In [ ]:
X = np.expand_dims(np.array(inter_vars), axis=1)

In [ ]:
W = weight.detach().cpu().numpy()

In [ ]:
b = bias.detach().cpu().numpy()
b

In [ ]:
mat_res = (W @ X).squeeze(axis=1) + b

In [ ]:
replace_dict = {}
for curr_expr, prev_var in zip(mat_res, prev_inter_vars):
    replace_dict[prev_var] = curr_expr
replace_dict

In [ ]:
prev_expression = expression
for key in replace_dict:
    expression = expression.subs(key, replace_dict[key])

In [ ]:
str(expression)

In [ ]:
variables, func_expr = func.expression()

In [ ]:
func_expr

### (Finally..) prune the box

In [ ]:
expr_nn = str(expression).replace("Max", "max")

In [ ]:
expr_nn

In [ ]:
variables

In [ ]:
def expression_squared_diff(expr1, expr2):
    return "(({}) - ({})) ^ 2".format(expr1, expr2)

In [ ]:
def expression_absolute_diff(expr1, expr2):
    return "abs(({}) - ({}))".format(expr1, expr2)

In [ ]:
def expression_add_const(expr, const):
    return "({}) + ({})".format(expr, const)

In [ ]:
ibex_expression = expression_squared_diff(func_expr, expr_nn)

In [ ]:
ibex_expression = expression_absolute_diff(func_expr, expr_nn)

In [ ]:
ibex_expression = expression_add_const(ibex_expression, -45)

In [ ]:
ibex_expression

In [ ]:
f = Function(*variables, ibex_expression)
X_in = IntervalVector(Interval(-10, 10))

In [ ]:
ctc = CtcFwdBwd(f, GEQ)  # root is when f = 0
ctc.contract(X_in)

In [ ]:
X_in